In [28]:
from IPython.core.display import display     # importing packages
from matplotlib import pyplot as plt
from numpy import linspace
from pandas import DataFrame
import pyomo.environ as pe
import pyomo.opt as po

## Demand Variables
# in kw
ED_LIGHTING = 200
ED_REFRIDGE = 1000
ED_HEATING = 100

## Electrical Effiency Variables
EE_ST = 0.68
EE_CHP = 0.44
EE_PV = 0.09
EE_WT = 0.22

## Heating Effiency Variables
HE_ST = 0
HE_CHP = 0.28
HE_PV = 0
HE_WT = 0

## Output Cost Variables
# $/kw
CC_LED = 10
CC_REFRIDGE = 70
CC_HEATING = 30

# $/kw * year
OC_LED = 1
OC_REFRIDGE = 4
OC_HEATING = 3

## Electricity Cost Variables
# $/kw
CC_ST = 250
CC_CHP = 500
CC_PV = 2000
CC_WT = 2000

# $/kw * year
OC_ST = 15
OC_CHP = 15
OC_PV = 500
OC_WT = 1200

## Output Efficiency Variables
EF_REFRIDGE = 3.0
EF_LED = 0.8
EF_HEATING = 0.85

## Input Prices
# $/GJ
IP_NG = 8.89
IP_BIOMASS = 9.72
IP_GRID = 36.11

## Time and Conversions
t_yrs = 20
t_hrs = 365*24*t_yrs
kwh_con = 277.78

m = pe.ConcreteModel()                      # creating the model

## CONTINUOUS VARIABLES
# in kw
m.Xc = pe.Var(domain=pe.NonNegativeReals)   # natural gas CHP
m.Xs = pe.Var(domain=pe.NonNegativeReals)   # biomass ST
m.Xp = pe.Var(domain=pe.NonNegativeReals)   # solar PV
m.Xw = pe.Var(domain=pe.NonNegativeReals)   # wind farm
m.Xg = pe.Var(domain=pe.NonNegativeReals)   # grid electricity
m.Xr = pe.Var(domain=pe.NonNegativeReals)   # refrigeration (output)
m.Xl = pe.Var(domain=pe.NonNegativeReals)   # lighting (output)
m.Xf = pe.Var(domain=pe.NonNegativeReals)   # heating (output)
m.Xn = pe.Var(domain=pe.NonNegativeReals)   # natural gas (raw input)
m.Xb = pe.Var(domain=pe.NonNegativeReals)   # biomass (raw input)

## BINARY VARIABLEs
m.Yc = pe.Var(domain=pe.Binary)   # natural gas CHP
m.Ys = pe.Var(domain=pe.Binary)   # biomass ST
m.Yp = pe.Var(domain=pe.Binary)   # solar PV
m.Yw = pe.Var(domain=pe.Binary)   # wind farm

## CONSTRAINTS
def e_mass_balance(m):
    return m.Xc*(EE_CHP/(EE_CHP + HE_CHP)) + m.Xs*EE_ST + m.Xp*EE_PV + m.Xw*EE_WT + m.Xg >= m.Xr + m.Xl
m.e_mass_balance = pe.Constraint(rule = e_mass_balance)

def h_mass_balance(m):
    return m.Xc*(HE_CHP/(HE_CHP + EE_CHP)) >= m.Xf
m.h_mass_balance = pe.Constraint(rule = h_mass_balance)

def L_demand(m):                                 # lighting demand
    return m.Xl*EF_LED >= ED_LIGHTING
m.L_demand = pe.Constraint(rule = L_demand)

def R_demand(m):                                 # refrigeration demand
    return m.Xr*EF_REFRIDGE >= ED_REFRIDGE
m.R_demand = pe.Constraint(rule = R_demand)

def H_demand(m):                                 # heating demand
    return m.Xf*EF_HEATING >= ED_HEATING
m.H_demand = pe.Constraint(rule = H_demand)

def NG_input(m):                                 # natural gas input balance
    return m.Xn*(EE_CHP + HE_CHP) >= m.Xc
m.NG_input = pe.Constraint(rule = NG_input)

def B_input(m):                                  # biomass input balance
    return m.Xb*EE_ST >= m.Xs
m.B_input = pe.Constraint(rule = B_input)

def ST_upper(m):                                 # upper bound biomass ST
    return m.Xs <= 1e6*m.Ys
m.ST_upper = pe.Constraint(rule = ST_upper)

def ST_lower(m):                                 # lower bound biomass ST
    return m.Xs >= 100*m.Ys
m.ST_lower = pe.Constraint(rule = ST_lower)

def CHP_upper(m):                                # upper bound natural gas CHP
    return m.Xc <= 1e6*m.Yc
m.CHP_upper = pe.Constraint(rule = CHP_upper)

def CHP_lower(m):                                # lower bound natural gas CHP
    return m.Xc >= 800*m.Yc
m.CHP_lower = pe.Constraint(rule = CHP_lower)

def PV_upper(m):                                 # upper bound solar PV
    return m.Xp <= 300*m.Yp
m.PV_upper = pe.Constraint(rule = PV_upper)

def PV_lower(m):                                 # lower bound solar PV
    return m.Xp >= 10*m.Yp
m.PV_lower = pe.Constraint(rule = PV_lower)

def WF_upper(m):                                 # upper bound wind farm
    return m.Xw <= 500*m.Yw
m.WF_upper = pe.Constraint(rule = WF_upper)

def WF_lower(m):                                 # lower bound wind farm
    return m.Xw >= 10*m.Yw
m.WF_lower = pe.Constraint(rule = WF_lower)

## OBJECTIVE FUNCTION
def objective_rule_efficiency(m):
    # sum of (output capacities)
    E_EFFICIENCY = m.Xl + m.Xr + m.Xf
    return E_EFFICIENCY
m.objective = pe.Objective(rule=objective_rule_efficiency, sense=pe.minimize)

solver = po.SolverFactory('glpk')
solution = solver.solve(m)

## COST EQUATION (for comparison)
CC_GEN = CC_ST*m.Xs.value + CC_CHP*m.Xc.value + CC_PV*m.Xp.value + CC_WT*m.Xw.value        
OC_GEN = (OC_ST*m.Xs.value + OC_CHP*m.Xc.value + OC_PV*m.Xp.value + OC_WT*m.Xw.value)*t_yrs
CC_CON = CC_LED*m.Xl.value + CC_REFRIDGE*m.Xr.value + CC_HEATING*m.Xf.value
OC_CON = (OC_LED*m.Xl.value + OC_REFRIDGE*m.Xr.value + OC_HEATING*m.Xf.value)*t_yrs
OC_INPUT = ((IP_NG/kwh_con)*m.Xn.value + (IP_BIOMASS/kwh_con)*m.Xb.value + (IP_GRID/kwh_con)*m.Xg.value)*t_hrs
SYSTEM_COST = CC_GEN + OC_GEN + CC_CON + OC_CON + OC_INPUT

## EMISSIONS EQUATION (for comparison)
EMISSIONS = (56*m.Xn.value + 100*m.Xb.value + 90*m.Xg.value) * t_hrs / (1e6*kwh_con)

m.objective()

700.9803921568621

In [29]:
system_efficiency = m.objective() / (m.Xc.value + m.Xs.value + m.Xp.value + m.Xw.value)

In [33]:
print(' --- MINIMIZED OBJECTIVE --- ')
print('SYSTEM ENERGY EFFICIENCY:', round(system_efficiency, 2), '\n')
print(' --- COMPARATIVE VALUES --- ')
print('TOTAL SYSTEM COST:', round(SYSTEM_COST, 2), 'USD ')
print('TOTAL CO2 EMISSIONS:', round(EMISSIONS, 2), 'kw \n')
print(' --- SYSTEM EFFICIECNY COMPONENTS --- ')
print('TOTAL ENERGY GENERATION:', round(m.Xc.value + m.Xs.value + m.Xp.value + m.Xw.value, 2), 'kw')
print('OUTPUT ENERGY LOAD:', round(m.objective(),2), 'kw', '\n')
print(' --- ENERGY MIX --- ')
print('NG CHP CAPACITY (Xc):', round(m.Xc.value,2), 'kw')
print('BIOMASS ST CAPACITY (Xs):', round(m.Xs.value, 2), 'kw')
print('SOLAR PV CAPACITY (Xp):', m.Xp.value, 'kw')
print('WIND FARM CAPACITY (Xw):', m.Xw.value, 'kw')
print('GRID ELECTRICITY (Xg):', round(m.Xg.value, 2), 'kw')

 --- MINIMIZED OBJECTIVE --- 
SYSTEM ENERGY EFFICIENCY: 0.73 

 --- COMPARATIVE VALUES --- 
TOTAL SYSTEM COST: 8265321.49 USD 
TOTAL CO2 EMISSIONS: 46.83 kw 

 --- SYSTEM EFFICIECNY COMPONENTS --- 
TOTAL ENERGY GENERATION: 954.55 kw
OUTPUT ENERGY LOAD: 700.98 kw 

 --- ENERGY MIX --- 
NG CHP CAPACITY (Xc): 954.55 kw
BIOMASS ST CAPACITY (Xs): 0.0 kw
SOLAR PV CAPACITY (Xp): 0.0 kw
WIND FARM CAPACITY (Xw): 0.0 kw
GRID ELECTRICITY (Xg): 0.0 kw


In [32]:
m.display()

Model unknown

  Variables:
    Xc : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 954.545454545454 :  None : False : False : NonNegativeReals
    Xs : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    Xp : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    Xw : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    Xg : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    Xr : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 333.333333333333